In [2]:
import numpy as np
import ipywidgets as ipw

from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
output_notebook()

from ipywidgets import interact

from collections import OrderedDict
old_settings = np.seterr(over = 'ignore') #Ignore warnings about overflow data points


Loading BokehJS ...

# Theoretical Background for 1D Case

## Overview

The code aims to illustrate the dispersion relation, allowed energy states, and density of states for a 1D free particle system. In this system, we assume that the particle is free to move along a one-dimensional line without any external forces or potential energy fields acting upon it.

## Dispersion Relation

The dispersion relation links the wave vector $ k $ to the energy $ E $ of the particle. For a free particle in 1D, the dispersion relation is given by:

\[$
E(k) = \frac{\hbar^2 k^2}{2m}
$\]

where $ \hbar $ is the reduced Planck constant, $ k $ is the wave vector, and $ m $ is the mass of the particle.

## Allowed States

For a free particle, all states are "allowed," meaning that the particle can have any energy value given by the dispersion relation. We can visualize these allowed states by plotting the energy as a function of the wave vector $ k $.

## Density of States

The density of states $ g(E) $ gives the number of states per unit energy at a given energy $ E $. For a free particle in 1D, the density of states is independent of the energy and is given by:

\[$
g(E) = \frac{1}{\pi \hbar} \sqrt{\frac{2m}{E}}
$\]

Here, $ \pi $ is the mathematical constant Pi, $ \hbar $ is the reduced Planck constant, $ m $ is the mass of the particle, and $ E $ is the energy.

In summary, this Python notebook aims to provide an interactive visualization of these important physical concepts for a 1D free particle system.


In [3]:
m_1D = 9.109e-31
hbar_1D = 6.626e-34 / 2.0 / np.pi
size_1D = 100e-9
Emax_1D = 2.0
k0_1D = np.pi / size_1D

N_1D = int(np.sqrt((Emax_1D * 1.602e-19 * 2 * m_1D) / hbar_1D**2) / k0_1D)
k_1D = np.linspace(-int(np.sqrt((Emax_1D * 1.602e-19 * 2 * m_1D) / hbar_1D**2)) / 1e9,
                   int(np.sqrt((Emax_1D * 1.602e-19 * 2 * m_1D) / hbar_1D**2)) / 1e9, 2 * N_1D)
En_1D = 6.242e+18 * (hbar_1D * k_1D * 1e9)**2 / (2 * m_1D)

In [4]:
freeE_1D = figure(height=350, width=300, title="Dispersion relationship of a free electron",
              tools="pan,reset,save,wheel_zoom", x_range=[np.amin(k_1D), np.amax(k_1D)],
              y_range=[0, 1.1 * np.amax(En_1D)])
freeE_1D.xaxis[0].axis_label = 'k (nm-1)'
freeE_1D.yaxis[0].axis_label = 'Energy (eV)'

source_1D = ColumnDataSource(data={'xVal_1D': k_1D, 'yVal_1D': En_1D})
freeE_1D.circle('xVal_1D', 'yVal_1D', source=source_1D, line_width=1, line_alpha=0.2)

bWidth_1D = 0.02
histoN_1D = (np.amax(En_1D) / bWidth_1D).astype(int)
kCount_1D = np.empty(histoN_1D, dtype=np.int8)
Ens_1D = np.arange(histoN_1D) * bWidth_1D
zero_1D = np.zeros(histoN_1D)

for idx in range(histoN_1D):
    kCount_1D[idx] = len(En_1D[(En_1D > idx * bWidth_1D) & (En_1D <= (idx + 1) * bWidth_1D)])

histo_1D = figure(height=350, width=300, title="allowed states per Energy interval",
              tools="pan,reset,save,wheel_zoom", y_range=[0, 1.1 * np.amax(En_1D)],
              x_range=[0, 1.1 * np.amax(kCount_1D)])

histo_1D.xaxis[0].axis_label = 'Number of states'
histo_1D.yaxis[0].axis_label = 'Energy (eV)'

source1_1D = ColumnDataSource(data={'bottom_1D': Ens_1D, 'top_1D': Ens_1D + bWidth_1D, 'right_1D': kCount_1D, 'left_1D': zero_1D})
histo_1D.quad(bottom='bottom_1D', top='top_1D', right='right_1D', left='left_1D', source=source1_1D,
           fill_color="navy", line_color="white", alpha=1)


# Calculate Density of states
gE_1D = 1/(np.pi*hbar_1D)*np.sqrt(m_1D/(2*1.602e-19*En_1D))
gEplot_1D = figure(height=350, width=300, title="Density of States (1D)",
              tools="pan,reset,save,wheel_zoom", x_range=[0.5*np.amin(gE_1D), 5*np.amin(gE_1D)], y_range=[0, 1.1*np.amax(En_1D)])
gEplot_1D.xaxis[0].axis_label = 'Density of States'
gEplot_1D.xaxis[0].formatter.precision = 2
gEplot_1D.xaxis[0].ticker.desired_num_ticks = 5
gEplot_1D.yaxis[0].axis_label = 'Energy (eV)'

source2_1D = ColumnDataSource(data={'xVal_1D': gE_1D, 'yVal_1D': En_1D})
gEplot_1D.circle('xVal_1D', 'yVal_1D', source=source2_1D, line_width=1, line_alpha=0.2)


# Set up callbacks to live update the plots
def update_data_1D(size_1D, Ntot_1D):
    k0_1D = np.pi / (size_1D)
    Emax_1D = 2 * (hbar_1D * k0_1D * Ntot_1D)**2 / m_1D / 1.602e-19
    N_1D = Ntot_1D  
    k_1D = np.linspace(-N_1D * k0_1D, N_1D * k0_1D, 2 * N_1D)
    En_1D = 6.242e+18 * (hbar_1D * k_1D * 1e9)**2 / (2 * m_1D)
    
    source_1D.data = {'xVal_1D': k_1D, 'yVal_1D': En_1D}

    histoN_1D = (np.amax(En_1D) / bWidth_1D).astype(int)
    kCount_1D = np.empty(histoN_1D, dtype=np.int8)
    Ens_1D = np.arange(histoN_1D) * bWidth_1D
    zero_1D = np.zeros(histoN_1D)
    for idx in range(histoN_1D):
        kCount_1D[idx] = len(En_1D[(En_1D > idx * bWidth_1D) & (En_1D <= (idx + 1) * bWidth_1D)])
    source1_1D.data = {'bottom_1D': Ens_1D, 'top_1D': Ens_1D + bWidth_1D, 'right_1D': kCount_1D, 'left_1D': zero_1D}

    gE_1D = 1 / (np.pi * hbar_1D) * np.sqrt(m_1D / (2 * 1.602e-19 * En_1D))
    source2_1D.data = {'xVal_1D': gE_1D, 'yVal_1D': En_1D}

    push_notebook(handle=handle_1D)

handle_1D = show(row(freeE_1D, histo_1D, gEplot_1D), notebook_handle=True)
interact(update_data_1D, size_1D=ipw.FloatSlider(min=2, max=202, step=10, value=50, description='Size (L, nm)'), Ntot_1D=ipw.IntSlider(min=4, max=604, step=10, value=200, description='No. Electrons'))

interactive(children=(FloatSlider(value=50.0, description='Size (L, nm)', max=202.0, min=2.0, step=10.0), IntS…

<function __main__.update_data_1D(size_1D, Ntot_1D)>

# Theoretical Background for 2D Case


## Dispersion Relation

For a free particle in two dimensions, the dispersion relation, which connects the wave vectors $ k_x $ and $ k_y $ to the energy $ E $, is given by:

\[$
E(k_x, k_y) = \frac{\hbar^2}{2m} (k_x^2 + k_y^2)
$\]

Here, $ \hbar $ is the reduced Planck constant, $ k_x $ and $ k_y $ are the components of the wave vector in the $ x $ and $ y $ directions, and $ m $ is the particle's mass.

## Allowed States

In a free particle system, all states are allowed, and the particle can occupy any point in $ k $-space corresponding to its energy as per the dispersion relation. These allowed states can be visualized in $ k $-space as points lying within a circle of radius \($ \sqrt{2mE}/\hbar $\).

## Density of States

The density of states $ g(E) $ describes the number of states per unit energy at a given energy $ E $. In 2D, the density of states is constant and is given by:

\[$
g(E) = \frac{m}{\pi \hbar^2}
$\]

This is a constant and is not dependent on the energy $ E $.

In summary, the Python notebook offers an interactive visualization of these essential physical principles for a 2D free particle system.


In [5]:
import numpy as np
import ipywidgets as ipw

from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
output_notebook()

from ipywidgets import interact

from collections import OrderedDict
old_settings = np.seterr(over = 'ignore') #Ignore warnings about overflow data points

# Set up constants and parameters_2D
m_2D = 9.109e-31  # Define electron mass
hbar_2D = 6.626e-34 / 2.0 / np.pi  # Reduced Planck constant
size_2D = 1e-9  # Define the width of the potential well in real space
Emax_2D = 2.0  # Pick the highest energy (eV) the electron could have by distribution
k0_2D = np.pi / size_2D  # Lowest allowed k in reciprocal space

# Set up the variable space_2D
kF_2D = np.sqrt(Emax_2D * 1.602e-19 * 2 * m_2D) / hbar_2D  # Calculate the Fermi wavevector
N_2D = int(np.pi * kF_2D ** 2 / k0_2D ** 2)  # calculate the number of allowed states by the energy constraint
x_2D = np.linspace(-N_2D * k0_2D, N_2D * k0_2D, 2 * N_2D, endpoint=False)  # Double the variable space by symmetry
y_2D = np.linspace(-N_2D * k0_2D, N_2D * k0_2D, 2 * N_2D, endpoint=False)  # Double the variable space by symmetry
kx_2D, ky_2D = np.meshgrid(x_2D, y_2D)
En_2D = (hbar_2D * x_2D * 1e9) ** 2 / (2 * m_2D)  # Energy for a free electron in eV

# Plot the dispersion relation of the free electrons
freeE_2D = figure(height=300, width=320, title="Dispersion relationship of a free electron",
              tools="pan,reset,save,wheel_zoom", x_range = [x_2D.min(), x_2D.max()],
              y_range = [0,1.1*En_2D.max()]);

freeE_2D.xaxis[0].axis_label='k (nm-1)';
freeE_2D.xaxis[0].formatter.precision = 1;
freeE_2D.xaxis[0].ticker.desired_num_ticks = 4;
freeE_2D.yaxis[0].axis_label='Energy (eV)';
freeE_2D.xaxis[0].formatter.precision = 1;

source0_2D = ColumnDataSource(data = {'xVal':kx_2D, 'yVal':En_2D});
freeE_2D.circle('xVal', 'yVal', source = source0_2D, line_width=1, line_alpha=0.2);

sourceC_2D = ColumnDataSource(data = {'x':[x_2D.min(), x_2D.max()],'y':[En_2D[N_2D-5], En_2D[N_2D+5]], 'label':[f'Ef = {En_2D[N_2D-5]:.2f} eV']*2});
freeE_2D.line(x = 'x', y = 'y', legend_field = 'label', source = sourceC_2D, line_color = 'red', line_alpha = 1.0);

# Plot the states per energy interval
states_2D = figure(height=300, width=350, title="Allowed states in k-space",
              tools="reset,pan,wheel_zoom", x_range = [1.1*x_2D.min(), 1.1*x_2D.max()],
              y_range = [1.1*y_2D.min(),1.1*y_2D.max()]);
states_2D.xaxis[0].axis_label='kx';
states_2D.xaxis[0].formatter.precision = 1;
states_2D.xaxis[0].ticker.desired_num_ticks = 3;
states_2D.yaxis[0].axis_label='ky';
states_2D.yaxis[0].formatter.precision = 1;

source1_2D = ColumnDataSource(data = {'x': kx_2D,'y': ky_2D});

x0_2D = np.linspace(x_2D[N_2D-5], x_2D[N_2D+5], 11, endpoint=True)
y0_2D = np.sqrt(abs(x0_2D.max()**2 - x0_2D**2))
x0_2D = np.concatenate((x0_2D, x0_2D))
y0_2D = np.concatenate((y0_2D, -y0_2D))

sourceC1_2D = ColumnDataSource(data = {'r': [x_2D[N_2D+5]-k0_2D/2], 'rr': [x_2D[N_2D+5]+k0_2D/2]})
sourceC2_2D = ColumnDataSource(data = {'x': x0_2D, 'y': y0_2D})
states_2D.rect('x', 'y', width=k0_2D, height=k0_2D, source=source1_2D, line_color='black', line_alpha=1.0, fill_alpha=0.0)
states_2D.annulus(x=0, y=0, inner_radius='r', outer_radius='rr', source=sourceC1_2D, line_color='red', line_alpha=1.0, fill_alpha=0.0)
states_2D.rect(x='x', y='y', width=k0_2D, height=k0_2D, source=sourceC2_2D, line_alpha=0.0, fill_alpha=1.0)

# Calculate Density of states
gE_2D = [m_2D / hbar_2D**2 / np.pi] * x_2D.size
gEplot_2D = figure(height=300, width=320, title="Density of States (2D)",
                   tools="pan,reset,save,wheel_zoom", x_range=[En_2D.min(), En_2D.max()], y_range=[0, 1.5*np.amax(gE_2D)])
gEplot_2D.xaxis[0].axis_label = 'Energy (eV)'
gEplot_2D.yaxis[0].axis_label = 'Density of States'
source2_2D = ColumnDataSource(data={'xVal': En_2D, 'yVal': gE_2D})
gEplot_2D.circle('xVal', 'yVal', source=source2_2D, line_width=1, line_alpha=0.2)

sourceC3_2D = ColumnDataSource(data = {'x': [En_2D[N_2D-5]], 'y': [gE_2D[N_2D-5]], 'label': [f'Ef = {En_2D[N_2D-5]:.2f} eV']})
gEplot_2D.circle('x', 'y', legend_field='label', source=sourceC3_2D, fill_color='red', fill_alpha=1.0, line_width=1.5, line_alpha=1.0, line_color='red')

# Set up callbacks to live update the plots
def update_data_2D(n_2D):
    # Generate the new curve
    currenE_2D = [f'Ef = {En_2D[N_2D + n_2D]:.2f} eV']
    sourceC_2D.data = {'x': [x_2D.min(), x_2D.max()], 'y': [En_2D[N_2D - n_2D], En_2D[N_2D + n_2D]], 'label': currenE_2D * 2}

    x0_2D = np.linspace(x_2D[N_2D - n_2D], x_2D[N_2D + n_2D], 2 * n_2D + 1, endpoint=True)
    y0_2D = np.sqrt(abs(x0_2D.max()**2 - x0_2D**2))
    x0_2D = np.concatenate((x0_2D, x0_2D))
    y0_2D = np.concatenate((y0_2D, -y0_2D))

    sourceC1_2D.data = {'r': [x_2D[N_2D + n_2D] + k0_2D / 2], 'rr': [x_2D[N_2D + n_2D] - k0_2D / 2]}
    sourceC2_2D.data = {'x': x0_2D, 'y': y0_2D}
    source2_2D.data = {'xVal': En_2D, 'yVal': gE_2D}
    sourceC3_2D.data = {'x': [En_2D[N_2D-n_2D]], 'y': [gE_2D[N_2D-n_2D]], 'label': [f'Ef = {En_2D[N_2D-n_2D]:.2f} eV']}
    
    push_notebook(handle=handle_2D)

handle_2D = show(row(freeE_2D, states_2D, gEplot_2D), notebook_handle=True)
interact(update_data_2D, n_2D=ipw.IntSlider(min=5, max=14, step=1, value=5, description='k/k0_2D'));

Loading BokehJS ...

interactive(children=(IntSlider(value=5, description='k/k0_2D', max=14, min=5), Output()), _dom_classes=('widg…

# Theoretical Background for 3D Case

## Dispersion Relation

The dispersion relation for a free particle in a 3D system can be expressed as:

\[$
E(k_x, k_y, k_z) = \frac{\hbar^2}{2m} (k_x^2 + k_y^2 + k_z^2)
$\]

Here, $ \hbar $ is the reduced Planck constant, \($ k_x, k_y, k_z $\) are the components of the wave vector in the $ x, y $ and $ z $ directions, respectively, and $ m $ is the mass of the particle.

## Allowed States

In a system of free particles, all states are generally allowed. These states can be visualized in $ k $-space as points within a sphere of radius \($ \sqrt{2mE}/\hbar $\).

## Density of States

The density of states $ g(E) $ describes the number of states per unit energy at a given energy $ E $. For a 3D system, the density of states is proportional to the square root of the energy and is given by:

\[$
g(E) = \frac{(2m)^{3/2}}{2\pi^2 \hbar^3} \sqrt{E}
$\]

This provides a quantitative measure of the number of states available for a particle to occupy at each energy level in a 3D system.


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Checkbox
from mpl_toolkits.mplot3d import Axes3D

# Constants
hbar = 1.0  # Reduced Planck's constant
m = 1.0     # Particle mass
k_range = 1.0  # Range for k values
N = 100  # Number of discrete kx values

# Define the dispersion relation for a free particle
def dispersion_relation(kx, ky, kz):
    return hbar**2 * (kx**2 + ky**2 + kz**2) / (2 * m)

# Linear dispersion relation
def linear_dispersion(kx, vF=1.0):
    return hbar * vF * kx

# Function to plot the E-kx relation
def plot_E_kx(Ef=0.2, linear=False):
    # Generate kx values
    kx = np.linspace(-k_range, k_range, N)
    
    # Determine the energy based on the dispersion relation
    E = linear_dispersion(kx) if linear else dispersion_relation(kx, 0, 0)
    
    # Create a figure
    plt.figure(figsize=(8, 6))
    
    # Plot the E-kx relation as discrete points or a line based on the dispersion relation
    plt.plot(kx, E, 'o' if not linear else '-', label=r'$E(k_x)$', color='blue', markersize=2)
    
    # Draw a horizontal line representing the Fermi energy level
    plt.axhline(y=Ef, color='r', linestyle='--', label=r'$E_F$')
    
    # Set labels and title
    plt.xlabel(r'$k_x$')
    plt.ylabel('Energy')
    plt.title(r'Dispersion Relation ($E$ vs $k_x$)')
    plt.legend()
    
    # Show the plot
    plt.grid(True)
    plt.show()

# Create an interactive plot with ipywidgets
interact(plot_E_kx, Ef=FloatSlider(value=0.2, min=0, max=2.0, step=0.01, description=r'$E_F$'))


interactive(children=(FloatSlider(value=0.2, description='$E_F$', max=2.0, step=0.01), Checkbox(value=False, d…

<function __main__.plot_E_kx(Ef=0.2, linear=False)>

In [6]:
# Function to plot the 3D allowed states
def plot_allowed_states(Ef=0.2):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    
    # Generate k values
    kx = np.linspace(-k_range, k_range, N)
    ky = np.linspace(-k_range, k_range, N)
    kz = np.linspace(-k_range, k_range, N)
    
    # Create a meshgrid for 3D plotting
    kx, ky, kz = np.meshgrid(kx, ky, kz)
    
    # Energy dispersion relation
    E = kx**2 + ky**2 + kz**2
    
    # Create a mask for allowed states (E < Ef)
    mask = E <= Ef
    
    # Plot allowed states
    ax.scatter(kx[mask], ky[mask], kz[mask], c='blue', alpha=0.6, s=20)
    
    # Set labels and title
    ax.set_xlabel(r'$k_x$')
    ax.set_ylabel(r'$k_y$')
    ax.set_zlabel(r'$k_z$')
    ax.set_title(r'Allowed states in $k$-space ($3D$)')
    
    # Set the same limits for all axes
    ax.set_xlim([-k_range, k_range])
    ax.set_ylim([-k_range, k_range])
    ax.set_zlim([-k_range, k_range])
    
    # Show the plot
    plt.show()


# Create an interactive plot with ipywidgets
interact(plot_allowed_states, Ef=FloatSlider(value=0.2, min=0, max=2.0, step=0.01, description=r'$E_F$'))


interactive(children=(FloatSlider(value=0.2, description='$E_F$', max=2.0, step=0.01), Output()), _dom_classes…

<function __main__.plot_allowed_states(Ef=0.2)>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Constants
hbar = 1.0  # Reduced Planck's constant
m = 1.0     # Particle mass

# Define the density of states for a 3D free electron gas
def density_of_states_3D(E):
    return (2*m)**(1.5) * np.sqrt(E) / (2*np.pi**2 * hbar**3)

# Function to plot the 3D DOS
def plot_DOS_3D(Ef=0.2):
    # Generate E values
    E = np.linspace(0, 2.0, 100)
    
    # Calculate DOS for each E
    gE = density_of_states_3D(E)
    
    # Create a figure
    plt.figure(figsize=(8, 6))
    
    # Plot the E-kx relation
    plt.plot(E, gE, label=r'$g(E)$', color='blue')
    
    # Draw a vertical line representing the Fermi energy level
    plt.axvline(x=Ef, color='r', linestyle='--', label=r'$E_F$')
    
    # Set labels and title
    plt.xlabel('Energy')
    plt.ylabel('Density of States')
    plt.title(r'Density of States ($3D$)')
    plt.legend()
    
    # Show the plot
    plt.grid(True)
    plt.show()

# Create an interactive plot with ipywidgets
interact(plot_DOS_3D, Ef=FloatSlider(value=0.2, min=0, max=2.0, step=0.01, description=r'$E_F$'))


interactive(children=(FloatSlider(value=0.2, description='$E_F$', max=2.0, step=0.01), Output()), _dom_classes…

<function __main__.plot_DOS_3D(Ef=0.2)>

# Self-Education Questions

## For the 1D Case

1. How does the dispersion relation of a free particle in 1D differ from that in 2D or 3D?
2. What does the density of states represent in the context of a 1D free particle?
3. How does the density of states in 1D compare to that in higher dimensions?

## For the 2D Case

1. Why is the density of states in 2D constant?
2. What geometric shape does the allowed states form in \( k \)-space in the 2D case?
3. How would you expect the dispersion relation to change if a potential was introduced?

## For the 3D Case

1. Explain how the density of states in 3D is derived.
2. How do the allowed states in \( k \)-space for the 3D case differ from the 2D and 1D cases?
3. What is the physical significance of a higher density of states at higher energy levels in a 3D system?